In [66]:
import redis
import faker as fk
import pandas
import random
fake = fk.Faker()

# main client
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

In [ ]:
def make_rand_computer():
    os = ['windows', 'macOS', 'linux']
    return {"ip": fake.ipv4(), "name": fake.user_name(), "os": random.choice(os)}

def make_rand_mac():
    return fake.mac_address()

def unravel_dict(dictionary):
    return " ".join([f"{key} {value}"for key,value in dictionary.items()])

def add_computer(computer):
    mac = make_rand_mac()
    r.execute_command(f"HSET mac:{mac} {unravel_dict(computer)}")
    r.execute_command(f"HSET index.mac.by.ip {computer['ip']} {mac}")
   
    macs = r.execute_command("HGETALL index.mac.by.ip")
    macs = macs[1::2]
    macs.remove(mac)
    for _ in range(4):
        if macs == []:
            break
        connected = random.choice(macs)
        r.execute_command(f"SADD connections:{mac} {connected}")
        r.execute_command(f"SADD connections:{connected} {mac}")
        print(mac)
        print(connected)

def initialize_content(n):
    for _ in range(n):
        add_computer(make_rand_computer())



1c:98:60:a4:df:25
be:ce:a3:08:f6:ed
1c:98:60:a4:df:25
be:ce:a3:08:f6:ed
1c:98:60:a4:df:25
be:ce:a3:08:f6:ed
1c:98:60:a4:df:25
be:ce:a3:08:f6:ed
1e:c2:00:07:02:c5
1c:98:60:a4:df:25
1e:c2:00:07:02:c5
be:ce:a3:08:f6:ed
1e:c2:00:07:02:c5
be:ce:a3:08:f6:ed
1e:c2:00:07:02:c5
1c:98:60:a4:df:25
d8:7a:42:f2:ad:68
be:ce:a3:08:f6:ed
d8:7a:42:f2:ad:68
be:ce:a3:08:f6:ed
d8:7a:42:f2:ad:68
1c:98:60:a4:df:25
d8:7a:42:f2:ad:68
1e:c2:00:07:02:c5
5a:82:cf:47:0d:6a
1e:c2:00:07:02:c5
5a:82:cf:47:0d:6a
be:ce:a3:08:f6:ed
5a:82:cf:47:0d:6a
be:ce:a3:08:f6:ed
5a:82:cf:47:0d:6a
d8:7a:42:f2:ad:68


In [109]:
for key in r.scan_iter("mac:*"):
    print(key)

mac:be:ce:a3:08:f6:ed
mac:1e:c2:00:07:02:c5
mac:1c:98:60:a4:df:25
mac:d8:7a:42:f2:ad:68
mac:5a:82:cf:47:0d:6a


In [110]:
r.flushdb()

True